In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from darts import TimeSeries

# Import Data

In [ ]:
df = pd.read_csv('csv/Ming12Jun67_V8_Group.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

# Data Preprocessing

In [ ]:
# แปลงคอลัมน์ order_completed_at เป็น datetime
df['order_completed_at'] = pd.to_datetime(df['order_completed_at'])

# Group by คอลัมน์ order_completed_at และนับจำนวน order ที่ไม่ซ้ำกัน
df_order = df.groupby(df['order_completed_at'])['number'].nunique().reset_index()

# เปลี่ยนชื่อคอลัมน์
df_order.columns = ['order_completed_at', 'unique_order_count']

# เรียงลำดับวันที่
df_order = df_order.sort_values('order_completed_at')

# แสดง DataFrame ที่ได้
print(df_order.head())
print(df_order.tail())

In [ ]:
df_order.info()

In [ ]:
df_order.head()

In [ ]:
df_order.to_csv('Ming12Jun67_total_order.csv', index=False)

In [ ]:
date_to_find = pd.Timestamp('2024-06-11').date()
df_order_filtered = df_order[df_order['order_completed_at'].dt.date == date_to_find]
df_order_filtered

In [ ]:
df_order = df_order[df_order['order_completed_at'] < '2024-06-12']

In [ ]:
df_order.info()

In [ ]:
import matplotlib.dates as mdates


series = TimeSeries.from_dataframe(df_order, 'order_completed_at', 'unique_order_count')

# ดูข้อมูลเบื้องต้นและปรับขนาดของภาพ
plt.figure(figsize=(20, 10))  # ปรับขนาดของภาพ (กว้าง x สูง)
series.plot()

# ตั้งค่ารูปแบบของวันที่บนแกน x
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gcf().autofmt_xdate()  # ปรับให้วันที่ไม่ซ้อนกัน

# กำหนด Title และชื่อของแกน x และ y
plt.title('Total Order By Date from 2022-2024')
plt.xlabel('Date')
plt.ylabel('Number of Orders (Unit)')

plt.show()

# Feature Engineering

# Modeling And Evaluate Model

In [ ]:
from darts.models import LinearRegressionModel, Prophet

In [ ]:
from darts.metrics import mape

# แบ่งข้อมูลออกเป็น train และ test set
train, val = series.split_before(0.9)
plt.figure(figsize=(20, 10))  # ปรับขนาดของภาพ (กว้าง x สูง)
train.plot(label="training")
val.plot(label="validation")


In [ ]:
# สร้างและฝึก Prophet Model
model_prophet = Prophet()
model_prophet.fit(train)

In [ ]:
# # สร้างและฝึก Linear Regression Model
# best_lag = 1
# for i in range(1,31):
#     model_lr = LinearRegressionModel(lags=i)
#     model_lr.fit(train)


#     # ทำนายค่าใน validation set
#     pred_lr = model_lr.predict(len(val))

#     # คำนวณค่า MAPE (Mean Absolute Percentage Error)
#     error_lr = mape(val, pred_lr)
#     if i==1:
#         error_best_lag = error_lr
#     else:
#         if error_lr < error_best_lag:
#             error_best_lag = error_lr; best_lag = i

# print(f'Linear Regression MAPE: lag {best_lag} {error_best_lag}')

In [ ]:
plt.figure(figsize=(20, 10))  # ปรับขนาดของภาพ (กว้าง x สูง)
pred_lr = model_prophet.predict(len(val))
pred_lr.plot(label="prediction")
train.plot(label="training")
val.plot(label="validation")

plt.legend()
plt.title('Train, Validation, and Prediction')
plt.xlabel('Date')
plt.ylabel('จำนวน Order หน่วย Unit')
plt.show()

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# from darts import TimeSeries
# from darts.models import LinearRegressionModel
# from darts.metrics import mape
# import matplotlib.dates as mdates



# # แบ่งข้อมูลออกเป็น train และ test set
# train, val = series.split_before(0.8)

# # สร้างฟังก์ชันสำหรับการทดสอบค่า lag ต่างๆ
# def find_best_lag(train, val, max_lag):
#     best_lag = 1
#     best_mape = float('inf')

#     for lag in range(1, max_lag + 1):
#         model_lr = LinearRegressionModel(lags=lag)
#         model_lr.fit(train)
#         pred_lr = model_lr.predict(len(val))
#         error_lr = mape(val, pred_lr)
#         print(f'Lag: {lag}, MAPE: {error_lr}')

#         if error_lr < best_mape:
#             best_mape = error_lr
#             best_lag = lag

#     return best_lag, best_mape

# # กำหนดค่า max_lag ที่ต้องการทดสอบ
# max_lag = 24

# # หา lag ที่ดีที่สุด
# best_lag, best_mape = find_best_lag(train, val, max_lag)

# print(f'Best Lag: {best_lag}, Best MAPE: {best_mape}')

In [ ]:
from darts.metrics import mape, mse, mae, rmse
model_prophet.fit(train)

# ทำนายค่าใน validation set
pred_prophet = model_prophet.predict(len(val))

# คำนวณค่า MAPE (Mean Absolute Percentage Error)
error_prophet = mape(val, pred_prophet)

print(f'Prophet MAPE: {error_prophet}')
print(f'MSE {mse(val, pred_lr)}')
print(f'MAE {mae(val, pred_lr)}')
print(f'RMSE {rmse(val, pred_lr)}')

In [ ]:
series.pd_dataframe().tail()

In [ ]:
# predict future
model_prophet.fit(series)

forecast = model_prophet.predict(7)
forecast = forecast.pd_dataframe().reset_index()
forecast.columns = ['Date', 'Total Order']
forecast.tail(7)

# Save Model and Load Model

In [ ]:
# from darts.models import LinearRegressionModel, Prophet

# model_prophet.save("Prophet_totalOrder_12Jun67.pkl")

In [ ]:
# model_loaded = Prophet.load("model/Prophet_totalOrder_12Jun67.pkl")